<a href="https://colab.research.google.com/github/Ajogeorge29/BONBID/blob/main/Track_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Track 1: Lesion Segmentation.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SimpleITK
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 15.6 MB/s eta 0:00:00


In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch.nn.functional as F

# Define data paths
adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/Preprocessed/adc"
z_adc_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/Preprocessed/z_adc"
label_dir = "/content/drive/MyDrive/BONBID CHALLENGE 2024/challenge harward/BONBID2023_Train/BONBID2023_Train/Preprocessed/label"

# Initialize dataset and data loader
dataset = BONBID_Dataset(adc_dir, z_adc_dir, label_dir, target_shape=(128, 224, 224))
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Test data loader
sample_inputs, sample_labels = next(iter(data_loader))
print("Sample input shape:", sample_inputs.shape)  # Expected: [batch_size, 2, 128, 224, 224]
print("Sample label shape:", sample_labels.shape)   # Expected: [batch_size, 1, 128, 224, 224]





Sample input shape: torch.Size([2, 2, 128, 224, 224])
Sample label shape: torch.Size([2, 1, 128, 224, 224])


In [ ]:
import numpy as np

# Find the maximum dimensions across the dataset
max_depth, max_height, max_width = 0, 0, 0

for base_name in sorted([f.replace('adc_resampled_', '').replace('.mha', '') for f in os.listdir(adc_dir)]):
    adc_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(adc_dir, f"adc_resampled_{base_name}.mha")))

    # Update maximum dimensions if the current image has larger dimensions
    max_depth = max(max_depth, adc_img.shape[0])
    max_height = max(max_height, adc_img.shape[1])
    max_width = max(max_width, adc_img.shape[2])

print("Maximum dimensions found:", max_depth, max_height, max_width)


Maximum dimensions found: 168 256 256


In [ ]:
class BONBID_Dataset(Dataset):
    def __init__(self, adc_dir, z_adc_dir, label_dir, transform=None, target_shape=(256, 380, 224)):
        self.adc_dir = adc_dir
        self.z_adc_dir = z_adc_dir
        self.label_dir = label_dir
        self.transform = transform
        self.target_shape = target_shape  # Fixed target shape
        self.file_names = sorted([f.replace('adc_resampled_', '').replace('.mha', '') for f in os.listdir(adc_dir)])

    def __len__(self):
        return len(self.file_names)

    def load_image(self, file_path):
        img = sitk.ReadImage(file_path)
        img_array = sitk.GetArrayFromImage(img).astype(np.float32)
        return img_array

    def resample_image(self, img, target_shape):
        img_tensor = torch.tensor(img)  # Convert to tensor
        # Use interpolation to resize to target shape
        img_resized = F.interpolate(img_tensor.unsqueeze(0).unsqueeze(0), size=target_shape, mode='trilinear', align_corners=False)
        return img_resized.squeeze(0).squeeze(0)  # Remove added dimensions

    def __getitem__(self, idx):
        base_name = self.file_names[idx]

        adc_path = os.path.join(self.adc_dir, f"adc_resampled_{base_name}.mha")
        z_adc_path = os.path.join(self.z_adc_dir, f"z_adc_resampled_{base_name}.mha")
        label_path = os.path.join(self.label_dir, f"label_resampled_{base_name}.mha")

        adc_img = self.load_image(adc_path)
        z_adc_img = self.load_image(z_adc_path)
        label_img = self.load_image(label_path)

        # Resample each image to the target shape
        adc_img = self.resample_image(adc_img, self.target_shape)
        z_adc_img = self.resample_image(z_adc_img, self.target_shape)
        label_img = self.resample_image(label_img, self.target_shape)

        # Stack ADC and Z_ADC images as channels
        input_img = torch.stack([adc_img, z_adc_img], dim=0)
        label_img = label_img.unsqueeze(0)  # Add channel dimension for the label

        return input_img, label_img


In [ ]:
# Initialize dataset and data loader with resampling
dataset = BONBID_Dataset(adc_dir, z_adc_dir, label_dir, target_shape=(256, 380, 224))
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Test data loader
sample_inputs, sample_labels = next(iter(data_loader))
print("Sample input shape:", sample_inputs.shape)  # Expected: [batch_size, 2, 256, 380, 224]
print("Sample label shape:", sample_labels.shape)   # Expected: [batch_size, 1, 256, 380, 224]


Sample input shape: torch.Size([2, 2, 256, 380, 224])
Sample label shape: torch.Size([2, 1, 256, 380, 224])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv3D(nn.Module):
    """(Conv3D => BatchNorm => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv3D, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()

        # Encoder
        self.enc1 = DoubleConv3D(in_channels, 64)
        self.enc2 = DoubleConv3D(64, 128)
        self.enc3 = DoubleConv3D(128, 256)
        self.enc4 = DoubleConv3D(256, 512)

        # Downsampling layers
        self.pool = nn.MaxPool3d(2)

        # Bottleneck
        self.bottleneck = DoubleConv3D(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose3d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = DoubleConv3D(1024, 512)

        self.upconv3 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.dec3 = DoubleConv3D(512, 256)

        self.upconv2 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.dec2 = DoubleConv3D(256, 128)

        self.upconv1 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.dec1 = DoubleConv3D(128, 64)

        # Output layer
        self.out_conv = nn.Conv3d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))

        # Decoder
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)

        # Output
        return self.out_conv(dec1)


In [ ]:
# Initialize the model
in_channels = 2  # Input channels (ADC and Z_ADC)
out_channels = 1  # Binary segmentation (lesion vs. non-lesion)

model = UNet3D(in_channels, out_channels)

# Print model summary
print(model)


UNet3D(
  (enc1): DoubleConv3D(
    (double_conv): Sequential(
      (0): Conv3d(2, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (enc2): DoubleConv3D(
    (double_conv): Sequential(
      (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (enc3): DoubleConv3D(
    (d

In [ ]:
import torch.optim as optim

# Loss function
criterion = nn.BCEWithLogitsLoss()  # Good for binary segmentation
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Dice Coefficient Metric

In [ ]:
def dice_coefficient(preds, targets, threshold=0.5):
    # Apply threshold to predictions
    preds = (preds > threshold).float()

    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum()
    dice = (2.0 * intersection + 1e-5) / (union + 1e-5)

    return dice


Training and Validation Loops

In [ ]:
import torch
import torch.optim as optim


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model, criterion, optimizer
model = UNet3D(in_channels=2, out_channels=1).to(device)
criterion = nn.BCEWithLogitsLoss()  # Use BCE with logits for binary segmentation
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training and validation functions
def train_one_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(data_loader)

def validate_one_epoch(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    dice_score = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            epoch_loss += loss.item()

            # Compute Dice coefficient
            dice_score += dice_coefficient(outputs.sigmoid(), labels).item()

    return epoch_loss / len(data_loader), dice_score / len(data_loader)


Using device: cpu


Setting Up Train and Validation Data Loaders

In [ ]:
from torch.utils.data import DataLoader, random_split

# Define the dataset
dataset = BONBID_Dataset(adc_dir, z_adc_dir, label_dir, target_shape=(256, 380, 224))

# Split into training and validation datasets (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)


NameError: name 'BONBID_Dataset' is not defined